# Tratativas no dataset

In [ ]:
import pandas as pd
import spacy

####Lendo o dataset e transformando em dois datasets: com e sem labels



In [ ]:
df_reviews = pd.read_csv("drive/MyDrive/Colab Notebooks/TCC/universal_studio_branches.csv")
df_reviews["written_date"] = pd.to_datetime(df_reviews["written_date"])
df_reviews["month"] = pd.DatetimeIndex(df_reviews["written_date"]).month
df_reviews["year"] = pd.DatetimeIndex(df_reviews["written_date"]).year
df_reviews["day_of_week"] = pd.DatetimeIndex(df_reviews["written_date"]).weekday

""" Reviews com rating 5 são considerados positivos e reviews com ratings 1 e 2 são considerados negativos
Isso se foi necessário, pois após análise do dataset, percebi que como não existe uma label neutra no dataset,
usar 3 e 4 como neutro exporia o modelo a muitas falhas. Assim, logo depois usei previsão para setar as labels
das reviews 3 e 4.
"""
# Setando as labels
df_reviews["label"] = df_reviews['rating'].apply(lambda x: 1 if x == 5 else None)
df_reviews.loc[df_reviews['rating'] <= 2, 'label'] = 0

#Criando os datasets
df_reviews_without_labels = df_reviews[df_reviews["label"].isnull()]
df_reviews_with_labels = df_reviews[~df_reviews["label"].isnull()]

In [ ]:
# Escrevendo datasets no drive
df_reviews_with_labels.to_csv("drive/MyDrive/Colab Notebooks/TCC/default_dataset.csv", index=False)
df_reviews_without_labels.to_csv("drive/MyDrive/Colab Notebooks/TCC/dataset_without_labels.csv", index=False)

In [ ]:
# Carregando linguagem do spacy inglês para ser utilizado no processo de tokenização
nlp_lemma = spacy.load('en_core_web_sm')


def convert_text_to_array(text):
    """Converte o texto para uma array utilizando o spacy"""
    text_converted = nlp_lemma(text)
    
    return [each_word for each_word in text_converted]


def get_token_and_lemma(text):
    """Gera os lemmas das palavras"""
    return [each_word.lemma_ for each_word in text]


def remove_stop_words(tokens):
    """Remove stop words, possíveis citações de usuários com @ e 
       mantém apenas os lemmas alfabéticos (remove pontuações)"""
    return [each_word for each_word in tokens \
            if not nlp_lemma.vocab[each_word].is_stop \
            and nlp_lemma.vocab[each_word].is_alpha \
            and each_word[:1] != "@"]

In [ ]:
"""Aplicando as tratativas nos dois datasets.
   Processo demorado, devido a quantidade de dados."""
df_reviews_with_labels['tokens'] = df_reviews_with_labels['review_text'].apply(lambda x: convert_text_to_array(x))
df_reviews_with_labels['tokens'] = df_reviews_with_labels['tokens'].apply(lambda x: get_token_and_lemma(x))
df_reviews_with_labels['tokens'] = df_reviews_with_labels['tokens'].apply(lambda x: remove_stop_words(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [ ]:
# Gerando datasets tratados
df_reviews_with_labels.to_csv("drive/MyDrive/Colab Notebooks/TCC/dataset_with_labels_and_tokens.csv")
df_reviews_without_labels.to_csv("drive/MyDrive/Colab Notebooks/TCC/dataset_without_labels_and_tokens.csv")